# VnV (ver. 2)

- JPark


In [1]:

#------------------------------------------------------
# Config
#------------------------------------------------------

# Test images

zip_images_url = 'http://keticmr.iptime.org:22080/edgeai/images/imagenet-mini-val.zip'
zip_images = 'imagenet-mini-val.zip'
dataset_root = './dataset'
fpath_zip_images = dataset_root + '/' + zip_images
fpath_testimages = dataset_root + '/imagenet-mini-val/'

# Models
#names = ['res18_weights.pth', 'res34_weights.pth', 'res50_weights.pth', 'res101_weights.pth', 'res152_weights.pth']
names = ['resnet18-dict.pth', 'resnet34-dict.pth', 'resnet50-dict.pth', 'resnet101-dict.pth', 'resnet152-dict.pth']
urlroot = 'http://keticmr.iptime.org:22080/edgeai/models_jpark/'
modeldir = './checkpoint/'

urlmodels = []
for name in names:
    urlmodels.append( urlroot + name )




In [2]:

#------------------------------------------------------
# Download data
#------------------------------------------------------
import os
# make download directory
def makedir(path): 
    try: 
        os.makedirs(path)
    except OSError: 
        if not os.path.isdir(path): 
            raise
    return os.path.abspath(path)

# Download images
import urllib, os

d = makedir(dataset_root) # 저장 공간 생성
url, fname = (zip_images_url, fpath_zip_images)
try: 
    urllib.URLopener().retrieve(url, fname)
except: 
    urllib.request.urlretrieve(url, fname)
    

# Unzip
cmd = 'unzip ' + fpath_zip_images + ' -d ' + dataset_root
print(cmd)
os.system(cmd)

# download models
makedir(modeldir)

    

unzip ./dataset/imagenet-mini-val.zip -d ./dataset
Archive:  ./dataset/imagenet-mini-val.zip


replace ./dataset/__MACOSX/._imagenet-mini-val? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


'/home/jpark/www/WorkDevEdgeAI/cloud-edge-aicontainers/v2/vnv/mission2/checkpoint'

In [3]:

import torch
for name in names:
    torch.hub.download_url_to_file(urlroot+name, modeldir+name)

## Run

In [4]:
#------------------------------------------------------
# Import packages
#------------------------------------------------------
import torch


model_fpaths = []
for name in names:
    model_fpaths.append(modeldir + name)

model_fpaths
#print(model_fpaths)
#print(urlroot+name)
#print(modeldir+name)


#------------------------------------------------------
# initial models
#------------------------------------------------------

model_names = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']
models = []

for model_name in model_names:
    m = torch.hub.load('pytorch/vision:v0.10.0', model_name, pretrained=False)
    models.append(m)
    m.eval()
    
# load saved weigts to the initial model
for idx, model in enumerate(models):
    checkpoint = urlmodels[idx]
    model.load_state_dict(torch.hub.load_state_dict_from_url(checkpoint, progress=False))
    model.eval()
    print(idx)
    
    

Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/jpark/.cache/torch/hub/pytorch_vision_v0.10.0


0
1
2
3
4


## Test data

In [5]:
from glob import iglob

'''
# read test files
testfiles = []
for fname in sorted( iglob(fpath_testimages + '**/*.JPEG', recursive=True) ):
    testfiles.append(fname)
'''

idx_gt = []
idx = 0
testfiles = []
for d in sorted( iglob(fpath_testimages + 'n*', recursive=False) ):
    for fname in sorted( iglob(d + '/*.JPEG', recursive=True) ):
        testfiles.append(fname)
        idx_gt.append( idx )
    idx += 1


# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]


preproc = ['method1', 'method2']
preproc_method = 'method1'

In [6]:
preproc[1]

'method2'

In [7]:
model_names

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import time
from tqdm import tqdm
import numpy as np

devices = ['cuda', 'cpu']
for device in devices: 
    print('-'*50)
    print('device = ', device, flush=True)
    print('-'*50)

    for model_idx, model in enumerate(models):
        start = time.time() # timer
        
        print(f'model = {model_names[model_idx]}', flush=True)
        top1_cnt = 0
        top5_cnt = 0

        testset = testfiles[:]
        n = len(testset)
        img_idx = 0
    
        # Define transforms for the evaluation phase
        if preproc_method == preproc[0]:
            preprocess = transforms.Compose([transforms.Resize(256),
                                                  transforms.CenterCrop(224),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                            ])
        else:
            preprocess = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

        for fpath in tqdm( testset ):
            #print( fpath )
            input_image = Image.open(fpath)

            try:
                input_tensor = preprocess(input_image)
            except:
                #print(img_idx)
                # gray scale to color
                input_image = Image.open(fpath).convert("RGB")
                input_tensor = preprocess(input_image)

            input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
            
            # move the input and model to GPU for speed if available
            if device == 'cuda':
                # move the input and model to GPU for speed if available
                if torch.cuda.is_available():
                    input_batch = input_batch.to('cuda')
                    model.to('cuda')
                else:
                    print('[-] cuda cannot available')
                    break
            else:
                input_batch = input_batch.to('cpu')
                model.to('cpu')


            with torch.no_grad():
                output = model(input_batch)

            # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
            #print(output[0])

            # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
            probabilities = torch.nn.functional.softmax(output[0], dim=0)
            #print(probabilities)

            # Show top categories per image
            top5_prob, top5_catid = torch.topk(probabilities, 5)
            for i in range(top5_prob.size(0)):
                #print(top5_catid[i])
                #pass
                #print(idx, ' ', categories[top5_catid[i]], top5_prob[i].item())

                if( top5_catid[i] == idx_gt[img_idx] ):
                    top1_cnt += 1

            #print('')
            img_idx += 1

        end = time.time()
        print('n = ', n)
        print('top1_cnt = ', top1_cnt)
        print('top1_cnt/n = ', top1_cnt/n)
        print('time = ', end - start)
        print('')

--------------------------------------------------
device =  cuda
--------------------------------------------------
model = resnet18


100%|██████████| 3923/3923 [00:46<00:00, 84.92it/s] 

n =  3923
top1_cnt =  3358
top1_cnt/n =  0.8559775681876115
time =  46.201945543289185

model = resnet34



100%|██████████| 3923/3923 [00:47<00:00, 82.91it/s] 

n =  3923
top1_cnt =  3476
top1_cnt/n =  0.8860565893448891
time =  47.3156373500824

model = resnet50


...


100%|██████████| 3923/3923 [03:24<00:00, 19.16it/s]

n =  3923
top1_cnt =  3545
top1_cnt/n =  0.9036451695131277
time =  204.8060154914856

model = resnet101



100%|██████████| 3923/3923 [05:14<00:00, 12.46it/s]

n =  3923
top1_cnt =  3572
top1_cnt/n =  0.9105276574050472
time =  314.93512892723083

model = resnet152



 56%|█████▌    | 2205/3923 [04:03<02:52,  9.99it/s]

  0%|          | 11/3923 [00:00<00:37, 104.65it/s]
--------------------------------------------------
device =  cuda
--------------------------------------------------
model = resnet18
100%|██████████| 3923/3923 [00:37<00:00, 103.82it/s]
  0%|          | 9/3923 [00:00<00:43, 89.72it/s]
n =  3923
top1_cnt =  3358
top1_cnt/n =  0.8559775681876115
time =  37.79109072685242
model = resnet34
100%|██████████| 3923/3923 [00:45<00:00, 85.61it/s]
  0%|          | 8/3923 [00:00<00:53, 72.95it/s]
n =  3923
top1_cnt =  3476
top1_cnt/n =  0.8860565893448891
time =  45.83013606071472
model = resnet50
100%|██████████| 3923/3923 [00:53<00:00, 73.31it/s]
  0%|          | 0/3923 [00:00<?, ?it/s]
n =  3923
top1_cnt =  3545
top1_cnt/n =  0.9036451695131277
time =  53.516403675079346
model = resnet101
100%|██████████| 3923/3923 [01:20<00:00, 48.47it/s]
  0%|          | 0/3923 [00:00<?, ?it/s]
n =  3923
top1_cnt =  3572
top1_cnt/n =  0.9105276574050472
time =  81.0622169971466
model = resnet152
100%|██████████| 3923/3923 [01:51<00:00, 35.12it/s]
  0%|          | 0/3923 [00:00<?, ?it/s]
n =  3923
top1_cnt =  3584
top1_cnt/n =  0.9135865409125669
time =  111.86518239974976